<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_BPR/test_sample_BPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Tue Jan 12 06:27:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# ! rm -rf PyTorch-Architectures/
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/modeling_BPR/

In [ ]:
! wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.train.rating
! wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.negative

In [3]:
import time
import pandas as pd
import numpy as np
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from model import BPR

In [4]:
dataset = pd.read_csv('ml-1m.train.rating', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

In [5]:
dataset.head()

,user_id,item_id,rating,timestamp
0,0,32,4,978824330
1,0,34,4,978824330
2,0,4,5,978824291
3,0,35,4,978824291
4,0,30,4,978824291


In [6]:
user_num = dataset['user_id'].max() + 1
item_num = dataset['item_id'].max() + 1

In [7]:
test_data = []
with open('ml-1m.test.negative', 'r') as fd:
  line = fd.readline()
  while line != None and line != '':
    arr = line.split('\t')
    u = eval(arr[0])[0]
    test_data.append([u, eval(arr[0])[1]])
    for i in arr[1:]:
      test_data.append([u, int(i)])
    line = fd.readline()

In [8]:
train_data = dataset.values.tolist()

In [9]:
train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)

In [10]:
for x in train_data:
  train_mat[x[0], x[1]] = 1.0

In [11]:
class RecDataset(Dataset):
  def __init__(self, features, num_item, train_mat=None, num_ng=4, is_training=False):
    self.features = features
    self.num_item = num_item
    self.train_mat = train_mat
    self.num_ng = num_ng
    self.is_training = is_training
    self.features_fill = []
    if self.is_training:
      self.negative_sample()
    
  def negative_sample(self):
    assert self.is_training, 'no need to sample while testing'
    for x in self.features:
      u, i = x[0], x[1]
      for t in range(self.num_ng):
        j = np.random.randint(self.num_item)
        while (u, j) in self.train_mat:
          j = np.random.randint(self.num_item)
        self.features_fill.append([u, i, j])
  
  def __len__(self):
    return self.num_ng * len(self.features) if self.is_training else len(self.features)
  
  def __getitem__(self, index):
    features = self.features_fill if self.is_training else self.features
    user = features[index][0]
    item_pos = features[index][1]
    item_neg = features[index][2] if self.is_training else features[index][1]
    return {
        'user_id': torch.tensor(user),
        'pos_item_id': torch.tensor(item_pos),
        'neg_item_id': torch.tensor(item_neg),
    }

In [12]:
train_dataset = RecDataset(train_data, num_item=item_num, train_mat=train_mat, is_training=True)
test_dataset = RecDataset(test_data, num_item=item_num, train_mat=train_mat, is_training=False)

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = BPR(n_users=user_num, n_items=item_num, embedding_size=64)
model.to(device)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total Trainable Parameters: ', params)

Total Trainable Parameters:  623744


In [14]:
# Hyperparameters
BATCH_SIZE_TRAIN = 2048
BATCH_SIZE_TEST = 100
EPOCHS = 3
LEARNING_RATE = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE_TEST, shuffle=False)

print('Length of Train Loader: ', len(train_loader))
print('Length of Test Loader: ', len(test_loader))

# Sanity check
for sample1, sample2 in zip(train_loader, test_loader):
  print(sample1.keys(), " ", sample2.keys())
  break

Length of Train Loader:  1942
Length of Test Loader:  6040
dict_keys(['user_id', 'pos_item_id', 'neg_item_id'])   dict_keys(['user_id', 'pos_item_id', 'neg_item_id'])


In [15]:
def hit(item, recommends):
  if item in recommends:
    return 1.
  return 0.

def calculate_hit_rate(model, data_loader, top_k, device):
  HR = []
  with torch.set_grad_enabled(False):
    for sample in data_loader:
      u_id = sample['user_id'].to(device)
      p_id = sample['pos_item_id'].to(device)
      n_id = sample['neg_item_id'].to(device) # not useful while testing

      interaction = {
          'user_id': u_id,
          'pos_item_id': p_id,
          'neg_item_id': n_id,
      }

      scores = model.predict(interaction)
      _, indices = torch.topk(scores, k=top_k)
      recommends = torch.take(p_id, indices)

      gt_item = p_id[0].item()
      HR.append(hit(gt_item, recommends))
  return torch.tensor(HR).mean().item()

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, sample in enumerate(train_loader):
    u_id = sample['user_id'].to(device)
    p_item = sample['pos_item_id'].to(device)
    n_item = sample['neg_item_id'].to(device)

    interaction = {
        'user_id': u_id,
        'pos_item_id': p_item,
        'neg_item_id': n_item,
    }

    optimizer.zero_grad()
    loss = model.calculate_loss(interaction=interaction)
    loss.backward()
    optimizer.step()

    # LOGGING
    if idx % 500 == 0:
      print('Epoch: %04d/%04d || Batch: %04d/%04d || Loss: %.3f' % (epoch+1, EPOCHS, idx, len(train_loader), loss.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    test_hr = calculate_hit_rate(model, test_loader, top_k=10, device=device)
    print('Test Hit Rate: ', test_hr)
  epoch_time = (time.time() - start_time) / 60
  print("Epoch Elapsed Time: %.2f min" % (epoch_time))
total_elapsed_time = (time.time() - start_time) / 60
print('Total Training Time: %.2f min' % (total_elapsed_time))

Epoch: 0001/0003 || Batch: 0000/1942 || Loss: 0.693
Epoch: 0001/0003 || Batch: 0500/1942 || Loss: 0.692
Epoch: 0001/0003 || Batch: 1000/1942 || Loss: 0.695
Epoch: 0001/0003 || Batch: 1500/1942 || Loss: 0.691
Test Hit Rate:  0.3889072835445404
Epoch Elapsed Time: 0.94 min
Epoch: 0002/0003 || Batch: 0000/1942 || Loss: 0.690
Epoch: 0002/0003 || Batch: 0500/1942 || Loss: 0.499
Epoch: 0002/0003 || Batch: 1000/1942 || Loss: 0.448
Epoch: 0002/0003 || Batch: 1500/1942 || Loss: 0.347
Test Hit Rate:  0.44387418031692505
Epoch Elapsed Time: 1.86 min
Epoch: 0003/0003 || Batch: 0000/1942 || Loss: 0.384
Epoch: 0003/0003 || Batch: 0500/1942 || Loss: 0.181
Epoch: 0003/0003 || Batch: 1000/1942 || Loss: 0.314
Epoch: 0003/0003 || Batch: 1500/1942 || Loss: 0.248
Test Hit Rate:  0.4596026539802551
Epoch Elapsed Time: 2.81 min
Total Training Time: 2.81 min
